In [2]:
import numpy as np
from tensorflow.keras.models import load_model
import librosa
from tensorflow.keras.preprocessing.sequence import pad_sequences

def load_and_preprocess_data(audio_path, label=None, n_mfcc=13, max_n_fft=2048):
    try:
        audio_signal, sample_rate = librosa.load(audio_path, sr=None)
        n_fft = min(max_n_fft, len(audio_signal))
        mfccs = librosa.feature.mfcc(y=audio_signal, sr=sample_rate, n_mfcc=n_mfcc, n_fft=n_fft)
        return mfccs.T, label
    except Exception as e:
        print(f"Error processing {audio_path}: {e}")
        return None, None

In [3]:
def preprocess_unseen_audio(audio_path, n_mfcc=13, max_n_fft=2048):
    features, label = load_and_preprocess_data(audio_path, label=None, n_mfcc=n_mfcc, max_n_fft=max_n_fft)
    if features is not None:
        return features
    else:
        print(f"Error processing {audio_path}")
        return None
    
model = load_model(r"real_batch_lstm_model_akn.h5")


def predict_audio(audio_path):
    processed_audio = preprocess_unseen_audio(audio_path)

    if processed_audio is not None:
        processed_audio = np.expand_dims(processed_audio, axis=0)
        processed_audio = pad_sequences(processed_audio, padding='post', dtype='float32')
        predictions = model.predict(processed_audio)
        predicted_class = np.argmax(predictions, axis=1)[0]

        if predicted_class == 0:
            print("The audio is predicted to be real.")
        else:
            print("The audio is predicted to be fake.")

        print("Prediction probabilities:", predictions)
    else:
        print("Error: Failed to preprocess audio.")

In [4]:
audio_file_path = r"C:\Users\akn\Desktop\Audio\fake\PA_E_2541211.flac"

predict_audio(audio_file_path)

1/1 [==============================] - 1s 825ms/step
The audio is predicted to be real.
Prediction probabilities: [[0.8107759  0.18922417]]
